In [1]:
import re
import os
import time
import random
import requests
import numpy as np
import pandas as pd
from os import system
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from selenium.webdriver.chrome.options import Options
import re

In [2]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 200)

#### Scrapes the best seller URLs from Amazon.com

In [12]:
def best_sellers_url():
    #webdriver to amazon best seller page 
    pd.set_option('display.max_columns', 20)
    pd.set_option('display.max_colwidth', 200)
    driver = webdriver.Chrome()
    driver.get('https://www.amazon.com/Best-Sellers/zgbs')
    
    best_sellers_total = []
    for i in range(40):
        x_path = driver.find_element_by_xpath(f'//*[@id="zg_browseRoot"]/ul/li[{i+1}]/a')
        url = x_path.get_attribute('href')
        best_sellers_total.append(url)
        
    driver.quit()
        
    return best_sellers_total
    

In [11]:
best_urls = best_sellers_url()

### Best Seller Item Scrape 
- Takes in the URL's from the above function 
- Scrapes the products from the amazon best seller lists 
- Scrapes URL's for each individual product to later feed into camelcamelcamel for price history
- creates list of ASIN numbers which will be used as the primary IDs in the DB 
- scrapes the ratings 
- names of the items 

In [50]:
def best_sellers_scrape(url):
    #webdriver to amazon best seller page 
    pd.set_option('display.max_columns', 20)
    pd.set_option('display.max_colwidth', 200)
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    #final Url list: outputs urls that will go to camelcamelcamel
    final_urls = []
    #will create list for primary keys 
    id_list = []
    #scrapes the name, ratings, reviews and price
    ratings = []
    item = driver.find_element_by_xpath('//*[@id="zg-center-div"]')
    items = item.text.split('#')
    final_list = []
#     [re.findall(r'\n(.*)',i)[:3] for i in items if re.findall(r'\n(.*)',i) != None and len(re.findall(r'\n(.*)',i))!= 0]
    for i in items:
        try:
            string_text = re.findall(r'\n(.*)',i)
            if re.findall(r'\n(.*)',i) != None and len(re.findall(r'\n(.*)',i))!= 0 :
                final_list.append(string_text[:3])
            for z in final_list:
                if z[2] == '':
                    z[2] = string_text[1]
                    z[1] = '000'
        except:
            continue
            
    link_list = [i.get_attribute('href') for i in driver.find_elements_by_class_name('a-link-normal')]
    
    for i in range(1,51):
        try:
            if driver.find_element_by_xpath(f'//*[@id="zg-ordered-list"]/li[{i}]/span/div/span/div[1]/a[1]'):
                rating = driver.find_element_by_xpath(f'//*[@id="zg-ordered-list"]/li[{i}]/span/div/span/div[1]/a[1]')
                ratings.append(rating.get_attribute('title'))
        except:
            continue


    #trys going to the second page if there is one.  If there are no best sellers it moves to the next page
    try:
        page_2 = driver.find_element_by_xpath('//*[@id="zg-center-div"]/div[2]/div/ul/li[3]/a')
    except:
        pass 
    try:
        page_2 = driver.find_element_by_class_name('a-normal')
    except:
        pass
    try:
        page_2.click()
    except:
        pass
   
    
    time.sleep(1.5)
    #scrapes the name, ratings, reviews and price for the second page 
    item = driver.find_element_by_xpath('//*[@id="zg-center-div"]')
    items = item.text.split('#')
    final_list1 = []
    
    for i in items:
        try:
            string_text = re.findall(r'\n(.*)',i)
            if re.findall(r'\n(.*)',i) != None and len(re.findall(r'\n(.*)',i))!= 0 :
                final_list1.append(string_text[:3])
            for z in final_list1:
                if z[2] == '':
                    z[2] = string_text[1]
                    z[1] = '000'
        except:
            continue
        
    
    link_list1 = [i.get_attribute('href') for i in driver.find_elements_by_class_name('a-link-normal')]
    #cleans up the links and appends them to a final url list 
    for i in link_list + link_list1:
        a = str(i)
        if a.find('reviews') == -1 and a not in final_urls and a.find('gp') == -1 and len(a) != 0:
            final_urls.append(a)
            
    #gets ids from urls and puts them in list id_list
    for z in final_urls:
        for i in range(len(z)-1):
            if z[i:i+4] == '/dp/':
                id_list.append(z[i+4:i+14])
                
    for i in range(1,51):
        try:
            rating = driver.find_element_by_xpath(f'//*[@id="zg-ordered-list"]/li[{i}]/span/div/span/div[1]/a[1]')
            ratings.append(rating.get_attribute('title'))
        except:
            continue
    
    
    
    driver.quit()
    return final_list + final_list1, final_urls, id_list, ratings

In [24]:
info, urls, ids, ratings = best_sellers_scrape(best_urls[6])

In [58]:
# def scrape_all():
#     info_all = []
#     urls_all = []
#     ids_all = []
#     ratings_all = []
#     for i in range(len(best_urls)):
#         try:
#             info, urls, ids, ratings = best_sellers_scrape(best_urls[i])
#             info_all.append(info)
#             urls_all.append(urls)
#             ids_all.append(ids)
#             ratings_all.append(ratings)
#         except:
#             continue

    
    

In [31]:
def zip_lists(info, urls, ids, ratings):
    new_list = []
    final_list = []
    for i in range(len(info)):
        new_list.append(info[i][0])
        new_list.append(info[i][1])
        new_list.append(info[i][2])
        new_list.append(ids[i])
        new_list.append(ratings[i][:3])
        new_list.append(urls[i])
        final_list.append(new_list)
        new_list = []
    return final_list

In [68]:
def scrape_all_zip():
    listy = []
    for i in range(len(best_urls)):
        try:
            info, urls, ids, ratings = best_sellers_scrape(best_urls[i])
            listy.append(zip_lists(info, urls, ids, ratings))
            
        except:
            continue
            
    return listy


In [69]:
amazon_item_data = scrape_all_zip()

In [83]:
def get_urls_for_camel(amazon_item_data):
    urls = []
    for category in amazon_item_data:
        for item in category:
            urls.append(item[-1])
            
    return urls

In [87]:
amazon_urls = get_urls_for_camel(amazon_item_data)